In [8]:
from google.colab import drive
import os
import glob


drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/Close')
data_dir = "Datas"
test_dir = "Test"
print("Data klasörü var mı?", os.path.exists(data_dir))
print("Test klasörü var mı?", os.path.exists(test_dir))
data_files = glob.glob(os.path.join(data_dir, "*.csv"))
print("Data CSV dosyaları:", data_files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data klasörü var mı? True
Test klasörü var mı? True
Data CSV dosyaları: ['Datas/ARCLK.csv', 'Datas/ASELS.csv', 'Datas/BIMAS.csv', 'Datas/EKGYO.csv', 'Datas/EREGL.csv', 'Datas/GARAN.csv', 'Datas/FROTO.csv', 'Datas/HEKTS.csv', 'Datas/GUBRF.csv', 'Datas/KCHOL.csv', 'Datas/KOZAA.csv', 'Datas/KOZAL.csv', 'Datas/KRDMD.csv', 'Datas/MGROS.csv', 'Datas/PGSUS.csv', 'Datas/SAHOL.csv', 'Datas/PETKM.csv', 'Datas/SASA.csv', 'Datas/SOKM.csv', 'Datas/SISE.csv', 'Datas/TCELL.csv', 'Datas/THYAO.csv', 'Datas/TOASO.csv', 'Datas/TKFEN.csv', 'Datas/VAKBN.csv', 'Datas/TUPRS.csv', 'Datas/BRSAN.csv', 'Datas/YKBNK.csv', 'Datas/ALARK.csv', 'Datas/AKBNK.csv']


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, log_loss, mean_squared_error
from sklearn.linear_model import (
    LogisticRegression,
    SGDClassifier,
    LinearRegression,
    SGDRegressor,
    HuberRegressor,
    PassiveAggressiveRegressor,
)
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from typing import Tuple, List, Dict
import ta
from sklearn.model_selection import TimeSeriesSplit
import copy
from datetime import timedelta
from copy import deepcopy

In [ ]:
'''CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 12,
}'''

'CONFIG = {\n    "window_size": 64,\n    "test_size": 64,\n    "data_paths": ["Datas/*.csv", "Test/*.csv"],\n    "start_date": "2006-07-27",\n    "end_date": "2025-05-06",\n    "n_splits": 12,\n}'

In [ ]:
'''def load_and_preprocess_all(paths: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:

    files = []
    company_index = CONFIG.get("company_index", None)

    datas_dir = "Datas"
    if not os.path.exists(datas_dir):
        print(f"Directory '{datas_dir}' not found. Please ensure your data is in the correct path.")
        raise FileNotFoundError(f"Directory '{datas_dir}' not found.")

    for pattern in paths:
        matched_files = sorted(glob.glob(pattern))
        if company_index is not None:
            if len(matched_files) > company_index:
                files.append(matched_files[company_index])
            else:
                print(f"Index {company_index} exceeds file count in path: {pattern}. Skipping this pattern.")
        else:
            files.extend(matched_files)

    if not files:
        raise FileNotFoundError("No CSV files found matching the patterns. Please check 'data_paths' and 'company_index'.")

    dfs = []
    for fn in files:
        try:
            df = pd.read_csv(
                fn, skiprows=2, header=None,
                names=["Date", "Close", "High", "Low", "Open", "Volume"]
            )
            df["Date"] = pd.to_datetime(df["Date"], errors='coerce', format='mixed')
            df = df.dropna(subset=["Date"])

            mask = (
                (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) &
                (df["Date"] <= pd.to_datetime(CONFIG["end_date"]))
            )
            df = df.loc[mask]

            if df.empty:
                print(f"DataFrame for {fn} is empty after date filtering. Skipping.")
                continue

            for col in ["Close", "High", "Low", "Open", "Volume"]:
                df[col] = pd.to_numeric(df[col], errors='coerce')

            df = df.dropna()

            if df.empty:
                print(f"DataFrame for {fn} is empty after NaN dropping. Skipping.")
                continue

            dfs.append(df)
        except Exception as e:
            print(f"Error processing {fn}: {e}")

    if not dfs:
        print("All dataframes are empty or failed to load. Returning empty DataFrames.")
        return pd.DataFrame(), pd.DataFrame()

    df = pd.concat(dfs, ignore_index=True)
    df = df.sort_values(by="Date")
    df = df.drop_duplicates(subset='Date', keep='last')

    date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='B')
    df = df.set_index('Date').reindex(date_range).rename_axis('Date').reset_index()

    df["Change"] = df["Close"].pct_change(fill_method=None) * 100
    df["Change"] = df["Change"].astype(float)
    df = df.dropna().reset_index(drop=True)

    df["CDirection"] = np.where(df["Change"] < 0, 0, 1)

    cd_count = df["CDirection"].value_counts()
    cd_pct   = df["CDirection"].value_counts(normalize=True) * 100
    print(f"CDirection count: {cd_count.to_dict()}")
    print(f"CDirection % distribution: {cd_pct.round(2).to_dict()}")

    if len(df) >= 20:
        df["MA_20"] = df["Close"].rolling(20).mean()
    else:
        print(f"DataFrame length ({len(df)}) is less than 20 for MA_20 calculation.")
        df["MA_20"] = df["Close"]

    if len(df) >= 14:
        df["RSI"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
    else:
        print(f"DataFrame length ({len(df)}) is less than 14 for RSI calculation.")
        df["RSI"] = 50.0

    if len(df) >= 5:
        df["Volatility"] = df["Close"].pct_change().rolling(5).std()
    else:
        print(f"DataFrame length ({len(df)}) is less than 5 for Volatility calculation.")
        df["Volatility"] = 0.01

    df = df.dropna().reset_index(drop=True)

    if df.empty:
        print("DataFrame is empty after feature engineering and dropping NaNs. Cannot proceed.")
        return pd.DataFrame(), pd.DataFrame()

    window_size = CONFIG["window_size"]
    test_size = CONFIG["test_size"]

    if len(df) < window_size + test_size:
        print(f"DataFrame length ({len(df)}) is very small for window_size ({window_size}). "
                       f"Consider increasing data duration. Current length: {len(df)}")
        test_size = max(int(len(df) * 0.2), 1)

    if len(df) >= window_size + test_size:
        df_test = df.iloc[-(test_size):].copy()
        df_train = df.iloc[:-(test_size)]
    else:
        print(f"DataFrame length ({len(df)}) is smaller than window_size+test_size. Using fallback split.")
        split_idx = max(len(df) - test_size, window_size)
        df_train = df.iloc[:split_idx].copy()
        df_test = df.iloc[split_idx:].copy()

    if df_train.empty:
        print("Training DataFrame is empty after splitting. Adjust 'start_date' or data length.")
        return pd.DataFrame(), pd.DataFrame()

    df_test
    df_train

    return df_train, df_test'''

'def load_and_preprocess_all(paths: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:\n\n    files = []\n    company_index = CONFIG.get("company_index", None)\n\n    datas_dir = "Datas"\n    if not os.path.exists(datas_dir):\n        print(f"Directory \'{datas_dir}\' not found. Please ensure your data is in the correct path.")\n        raise FileNotFoundError(f"Directory \'{datas_dir}\' not found.")\n\n    for pattern in paths:\n        matched_files = sorted(glob.glob(pattern))\n        if company_index is not None:\n            if len(matched_files) > company_index:\n                files.append(matched_files[company_index])\n            else:\n                print(f"Index {company_index} exceeds file count in path: {pattern}. Skipping this pattern.")\n        else:\n            files.extend(matched_files)\n\n    if not files:\n        raise FileNotFoundError("No CSV files found matching the patterns. Please check \'data_paths\' and \'company_index\'.")\n\n    dfs = []\n    for fn in

In [ ]:
'''def dater(df_train, df_test):
    df_train["Date"] = pd.to_datetime(df_train["Date"])
    df_train["Year"] = df_train["Date"].dt.year
    df_train["Month"] = df_train["Date"].dt.month
    df_train["Day"] = df_train["Date"].dt.day
    df_train["Weekday"] = df_train["Date"].dt.weekday
    df_train = df_train.drop(columns=["Date", "Change"], errors="ignore")

    X_train = df_train.drop("CDirection", axis=1)
    y_train = df_train["CDirection"]


    df_test["Date"] = pd.to_datetime(df_test["Date"])
    df_test["Year"] = df_test["Date"].dt.year
    df_test["Month"] = df_test["Date"].dt.month
    df_test["Day"] = df_test["Date"].dt.day
    df_test["Weekday"] = df_test["Date"].dt.weekday

    actual_cdirection = df_test["CDirection"].reset_index(drop=True) if "CDirection" in df_test.columns else None
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test.drop(columns=["Date", "Change", "CDirection"], errors="ignore")

    return X_train, y_train, actual_cdirection, test_dates, X_test'''

'def dater(df_train, df_test):\n    df_train["Date"] = pd.to_datetime(df_train["Date"])\n    df_train["Year"] = df_train["Date"].dt.year\n    df_train["Month"] = df_train["Date"].dt.month\n    df_train["Day"] = df_train["Date"].dt.day\n    df_train["Weekday"] = df_train["Date"].dt.weekday\n    df_train = df_train.drop(columns=["Date", "Change"], errors="ignore")\n\n    X_train = df_train.drop("CDirection", axis=1)\n    y_train = df_train["CDirection"]\n\n\n    df_test["Date"] = pd.to_datetime(df_test["Date"])\n    df_test["Year"] = df_test["Date"].dt.year\n    df_test["Month"] = df_test["Date"].dt.month\n    df_test["Day"] = df_test["Date"].dt.day\n    df_test["Weekday"] = df_test["Date"].dt.weekday\n\n    actual_cdirection = df_test["CDirection"].reset_index(drop=True) if "CDirection" in df_test.columns else None\n    test_dates = df_test["Date"].reset_index(drop=True)\n    X_test = df_test.drop(columns=["Date", "Change", "CDirection"], errors="ignore")\n\n    return X_train, y_train,

In [ ]:
'''def DRpredictor():

    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])


    X_train, y_train, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)


    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    all_results = {}


    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
        X_t, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]


        scaler = StandardScaler()
        X_t_scaled = scaler.fit_transform(X_t)
        X_v_scaled = scaler.transform(X_v)


        clf = LogisticRegression(random_state=42)
        clf.fit(X_t_scaled, y_t)


        tr_loss = log_loss(y_t, clf.predict_proba(X_t_scaled))
        vl_loss = log_loss(y_v, clf.predict_proba(X_v_scaled))
        print(f"Fold {fold} — Train Loss: {tr_loss:.4f}, Val Loss: {vl_loss:.4f}")


        all_results[fold] = {
            'model': clf,
            'scaler': scaler,
            'val_loss': vl_loss
        }


    best_fold = min(all_results, key=lambda f: all_results[f]['val_loss'])
    info = all_results[best_fold]
    print(f"\nEn iyi fold: {best_fold} — Val Loss = {info['val_loss']:.4f}")


    best_clf = info['model']
    best_scaler = info['scaler']


    print("\n=== Test Aşaması ===")
    X_test_scaled = best_scaler.transform(X_test)
    y_pred = best_clf.predict(X_test_scaled)


    df_result = pd.DataFrame({
        'Date': test_dates,
        'Real_CDirection': actual_cdirection,
        'Predicted_CDirection': y_pred
    })

    print(f"Toplam {len(y_pred)} tahmin yapıldı: {df_result['Date'].iloc[0]} → {df_result['Date'].iloc[-1]}")
    return df_result'''

'def DRpredictor():\n\n    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])\n\n\n    X_train, y_train, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)\n\n\n    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])\n    all_results = {}\n\n\n    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):\n        X_t, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]\n        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]\n\n\n        scaler = StandardScaler()\n        X_t_scaled = scaler.fit_transform(X_t)\n        X_v_scaled = scaler.transform(X_v)\n\n\n        clf = LogisticRegression(random_state=42)\n        clf.fit(X_t_scaled, y_t)\n\n\n        tr_loss = log_loss(y_t, clf.predict_proba(X_t_scaled))\n        vl_loss = log_loss(y_v, clf.predict_proba(X_v_scaled))\n        print(f"Fold {fold} — Train Loss: {tr_loss:.4f}, Val Loss: {vl_loss:.4f}")\n\n\n        all_results[fold] = {\n            \'model\': clf,\n           

In [ ]:
'''def DRpredictor():

    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])
    X_train_orig, y_train_orig, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)


    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    all_results = {}
    for fold, (tr_idx, val_idx) in enumerate(tscv.split(X_train_orig), 1):
        X_t, X_v = X_train_orig.iloc[tr_idx], X_train_orig.iloc[val_idx]
        y_t, y_v = y_train_orig.iloc[tr_idx], y_train_orig.iloc[val_idx]

        scaler = StandardScaler()
        X_t_s = scaler.fit_transform(X_t)
        X_v_s = scaler.transform(X_v)

        clf = LogisticRegression(random_state=42)
        clf.fit(X_t_s, y_t)
        vl_loss = log_loss(y_v, clf.predict_proba(X_v_s))

        all_results[fold] = {"model": clf, "scaler": scaler, "val_loss": vl_loss}

    best_fold = min(all_results, key=lambda f: all_results[f]["val_loss"])
    print(f"\nEn iyi fold: {best_fold} — Val Loss = {all_results[best_fold]['val_loss']:.4f}")




    X_train_wf = X_train_orig.copy()
    y_train_wf = y_train_orig.copy()

    results = {"Date": [], "Real_CDirection": [], "Predicted_CDirection": [], "Error_Rate": []}

    wrong_count = 0

    for i in range(len(X_test)):

        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_train_wf)
        clf = LogisticRegression(random_state=42)
        clf.fit(X_tr_s, y_train_wf)


        x_i = X_test.iloc[[i]]
        x_i_s = scaler.transform(x_i)
        y_hat = clf.predict(x_i_s)[0]
        y_true = actual_cdirection.iloc[i]
        dt = test_dates.iloc[i]

        if y_hat != y_true:
            wrong_count += 1
        error_rate = (wrong_count / (i + 1)) * 100


        results["Date"].append(dt)
        results["Real_CDirection"].append(y_true)
        results["Predicted_CDirection"].append(y_hat)
        results["Error_Rate"].append(error_rate)


        X_train_wf = pd.concat([X_train_wf, x_i], ignore_index=True)
        y_train_wf = pd.concat([y_train_wf, pd.Series([y_true])], ignore_index=True)


    df_result = pd.DataFrame(results)
    print(f"\nWalk-forward ile toplam {len(df_result)} tahmin yapıldı: "
          f"{df_result['Date'].iloc[0]} → {df_result['Date'].iloc[-1]}")
    return df_result'''


'def DRpredictor():\n\n    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])\n    X_train_orig, y_train_orig, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)\n\n\n    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])\n    all_results = {}\n    for fold, (tr_idx, val_idx) in enumerate(tscv.split(X_train_orig), 1):\n        X_t, X_v = X_train_orig.iloc[tr_idx], X_train_orig.iloc[val_idx]\n        y_t, y_v = y_train_orig.iloc[tr_idx], y_train_orig.iloc[val_idx]\n\n        scaler = StandardScaler()\n        X_t_s = scaler.fit_transform(X_t)\n        X_v_s = scaler.transform(X_v)\n\n        clf = LogisticRegression(random_state=42)\n        clf.fit(X_t_s, y_t)\n        vl_loss = log_loss(y_v, clf.predict_proba(X_v_s))\n\n        all_results[fold] = {"model": clf, "scaler": scaler, "val_loss": vl_loss}\n\n    best_fold = min(all_results, key=lambda f: all_results[f]["val_loss"])\n    print(f"\nEn iyi fold: {best_fold} — Val Loss = {all_results[best_

In [ ]:
'''def DRpredictor():

    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])
    X_train_orig, y_train_orig, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)


    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    cv_losses = []
    for tr_idx, val_idx in tscv.split(X_train_orig):
        X_t, X_v = X_train_orig.iloc[tr_idx], X_train_orig.iloc[val_idx]
        y_t, y_v = y_train_orig.iloc[tr_idx], y_train_orig.iloc[val_idx]
        scaler_cv = StandardScaler().fit(X_t)
        clf_cv    = LogisticRegression(random_state=42).fit(
                        scaler_cv.transform(X_t), y_t
                    )
        cv_losses.append(log_loss(y_v, clf_cv.predict_proba(scaler_cv.transform(X_v))))
    best_fold = int(pd.Series(cv_losses).idxmin()) + 1
    print(f"En iyi fold (bilgi amaçlı): {best_fold} — Val Loss = {cv_losses[best_fold-1]:.4f}")


    final_scaler = StandardScaler().fit(X_train_orig)
    final_clf    = LogisticRegression(random_state=42).fit(
                       final_scaler.transform(X_train_orig),
                       y_train_orig
                   )


    X_train_wf = X_train_orig.copy()
    y_train_wf = y_train_orig.copy()
    wrong_count = 0

    results = {
        "Date": [],
        "Real_CDirection": [],
        "Predicted_CDirection": [],
        "Error_Rate": []
    }


    for i in range(len(X_test)):

        x_i    = X_test.iloc[[i]]
        x_i_s  = final_scaler.transform(x_i)
        y_hat  = final_clf.predict(x_i_s)[0]
        y_true = actual_cdirection.iloc[i]
        dt     = test_dates.iloc[i]


        if y_hat != y_true:
            wrong_count += 1
        error_rate = wrong_count / (i + 1) * 100


        results["Date"].append(dt)
        results["Real_CDirection"].append(y_true)
        results["Predicted_CDirection"].append(y_hat)
        results["Error_Rate"].append(error_rate)


        X_train_wf = pd.concat([X_train_wf, x_i], ignore_index=True)
        y_train_wf = pd.concat([y_train_wf, pd.Series([y_true])], ignore_index=True)


    df_result = pd.DataFrame(results)
    print(f"\nWalk-forward süreci tamamlandı: {len(df_result)} adımdan "
          f"{df_result['Date'].iloc[0]} → {df_result['Date'].iloc[-1]}")
    return df_result'''


'def DRpredictor():\n\n    df_train_raw, df_test = load_and_preprocess_all(CONFIG["data_paths"])\n    X_train_orig, y_train_orig, actual_cdirection, test_dates, X_test = dater(df_train_raw, df_test)\n\n\n    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])\n    cv_losses = []\n    for tr_idx, val_idx in tscv.split(X_train_orig):\n        X_t, X_v = X_train_orig.iloc[tr_idx], X_train_orig.iloc[val_idx]\n        y_t, y_v = y_train_orig.iloc[tr_idx], y_train_orig.iloc[val_idx]\n        scaler_cv = StandardScaler().fit(X_t)\n        clf_cv    = LogisticRegression(random_state=42).fit(\n                        scaler_cv.transform(X_t), y_t\n                    )\n        cv_losses.append(log_loss(y_v, clf_cv.predict_proba(scaler_cv.transform(X_v))))\n    best_fold = int(pd.Series(cv_losses).idxmin()) + 1\n    print(f"En iyi fold (bilgi amaçlı): {best_fold} — Val Loss = {cv_losses[best_fold-1]:.4f}")\n\n\n    final_scaler = StandardScaler().fit(X_train_orig)\n    final_clf    = LogisticRe

In [ ]:
'''result_dir = "Results/Change"
os.makedirs(result_dir, exist_ok=True)


company_files = sorted(glob.glob("Datas/*.csv"))

for idx, filepath in enumerate(company_files):

    company_code = os.path.splitext(os.path.basename(filepath))[0]
    print(f"\n=== Processing [{idx}] {company_code} ===")


    CONFIG["company_index"] = idx


    result_df = DRpredictor()


    result_path = f"{result_dir}/{company_code}_results.csv"
    result_df.to_csv(result_path, index=False)
    print(f"→ Saved: {result_path}")'''

'result_dir = "Results/Change"\nos.makedirs(result_dir, exist_ok=True)\n\n\ncompany_files = sorted(glob.glob("Datas/*.csv"))\n\nfor idx, filepath in enumerate(company_files):\n\n    company_code = os.path.splitext(os.path.basename(filepath))[0]\n    print(f"\n=== Processing [{idx}] {company_code} ===")\n\n\n    CONFIG["company_index"] = idx\n\n\n    result_df = DRpredictor()\n\n\n    result_path = f"{result_dir}/{company_code}_results.csv"\n    result_df.to_csv(result_path, index=False)\n    print(f"→ Saved: {result_path}")'

In [ ]:
base_dir = "Results/Change"
methods = ["Vanilla", "Retrain", "Stepper", "Beast"]

summary = []

for method in methods:
    method_dir = os.path.join(base_dir, method)
    files = sorted(glob.glob(os.path.join(method_dir, "*_results.csv")))


    total_wrong = 0
    total_count = 0

    per_company = []

    for fn in files:
        df = pd.read_csv(fn)

        wrong = (df["Real_CDirection"] != df["Predicted_CDirection"]).sum()
        count = len(df)
        err_rate = wrong / count * 100

        total_wrong += wrong
        total_count += count

        company_code = os.path.splitext(os.path.basename(fn))[0].replace("_results", "")
        per_company.append({
            "Method": method,
            "Company": company_code,
            "Error_Rate (%)": err_rate
        })


    overall_err = total_wrong / total_count * 100 if total_count else None
    summary.append({
        "Method": method,
        "Overall_Error_Rate (%)": overall_err
    })


    df_pc = pd.DataFrame(per_company).set_index(["Method", "Company"])
    print(f"\n=== {method} — Şirket Bazında Hata Oranları ===")
    print(df_pc.to_string())


df_summary = pd.DataFrame(summary).set_index("Method")
print("\n=== Yöntem Geneli Hata Oranları ===")
print(df_summary.to_string())



=== Vanilla — Şirket Bazında Hata Oranları ===
                 Error_Rate (%)
Method  Company                
Vanilla AKBNK           20.3125
        ALARK           23.4375
        ARCLK           17.1875
        ASELS           23.4375
        BIMAS           18.7500
        BRSAN           17.1875
        EKGYO            9.3750
        EREGL           39.0625
        FROTO           65.6250
        GARAN           12.5000
        GUBRF           14.0625
        HEKTS           43.7500
        KCHOL           25.0000
        KOZAA           12.5000
        KOZAL           23.4375
        KRDMD           28.1250
        MGROS           17.1875
        PETKM           15.6250
        PGSUS           45.3125
        SAHOL           20.3125
        SASA             7.8125
        SISE            28.1250
        SOKM            14.0625
        TCELL           14.0625
        THYAO           20.3125
        TKFEN           32.8125
        TOASO           15.6250
        TUPRS           

In [ ]:
'''import os
import glob
import numpy as np
import pandas as pd
from typing import List, Tuple
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import log_loss
from datetime import timedelta


CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 12,
}

def load_and_preprocess_single(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(
            file_path,
            skiprows=2,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"]
        )


        df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
        df = df.dropna(subset=["Date"])


        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)
        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)
        df = df.loc[mask]

        if df.empty:
            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")
            return pd.DataFrame()


        for col in ["Close", "High", "Low", "Open", "Volume"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        return df.dropna()

    except Exception as e:
        print(f"Hata: {file_path} yüklenirken - {str(e)}")
        return pd.DataFrame()

def load_company_data(company_code: str) -> Tuple[pd.DataFrame, pd.DataFrame]:

    train_path = f"Datas/{company_code}.csv"
    test_path = f"Test/{company_code}.csv"


    train_df = load_and_preprocess_single(train_path)
    test_df = load_and_preprocess_single(test_path)

    if train_df.empty or test_df.empty:
        return pd.DataFrame(), pd.DataFrame()


    def calculate_features(df):
        df = df.copy()
        df["Change"] = df["Close"].pct_change(fill_method=None) * 100
        df["CDirection"] = np.where(df["Change"] < 0, 0, 1)


        df["MA_20"] = df["Close"].rolling(20, min_periods=1).mean()


        rsi = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
        df["RSI"] = rsi.fillna(50)


        volatility = df["Close"].pct_change().rolling(5, min_periods=1).std()
        df["Volatility"] = volatility.fillna(0.01)

        return df.dropna()


    train_df = calculate_features(train_df)
    test_df = calculate_features(test_df)

    return train_df, test_df

def dater(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple:
    def process_date_features(df):
        df = df.copy()
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Month"] = df["Date"].dt.month
        df["Day"] = df["Date"].dt.day
        df["Weekday"] = df["Date"].dt.weekday
        return df

    df_train = process_date_features(df_train)
    df_test = process_date_features(df_test)


    y_train = df_train["CDirection"]
    X_train = df_train.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")


    actual_cdirection = df_test["CDirection"].reset_index(drop=True)
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    return X_train, y_train, actual_cdirection, test_dates, X_test

def DRpredictor(company_code: str) -> pd.DataFrame:

    train_df, test_df = load_company_data(company_code)

    if train_df.empty or test_df.empty:
        print(f"{company_code} için yeterli veri yok. Atlanıyor.")
        return pd.DataFrame()

    X_train, y_train, actual_cdirection, test_dates, X_test = dater(train_df, test_df)


    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    best_val_loss = float('inf')
    best_scaler = None
    best_model = None

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):

        X_t, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]


        scaler = StandardScaler()
        X_t_scaled = scaler.fit_transform(X_t)
        X_v_scaled = scaler.transform(X_v)


        model = SGDClassifier(
            loss='log_loss',
            learning_rate='adaptive',
            eta0=0.01,
            max_iter=1000,
            tol=1e-3,
            random_state=42,
            early_stopping=True
        )
        model.fit(X_t_scaled, y_t)


        val_loss = log_loss(y_v, model.predict_proba(X_v_scaled))


        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_scaler = scaler
            best_model = model

    print(f"\nEn iyi model val_loss: {best_val_loss:.4f}")


    results = {
        "Date": [],
        "Real_CDirection": [],
        "Predicted_CDirection": [],
        "Error_Rate": [],
        "Cumulative_Accuracy": []
    }

    correct_count = 0
    X_walk = X_train.copy()
    y_walk = y_train.copy()


    dynamic_model = SGDClassifier(
        loss='log_loss',
        learning_rate='adaptive',
        eta0=0.01,
        max_iter=1000,
        warm_start=True,
        random_state=42
    )


    X_walk_scaled = best_scaler.transform(X_walk)
    dynamic_model.fit(X_walk_scaled, y_walk)

    for i in range(len(X_test)):

        x_i = X_test.iloc[[i]]
        x_i_scaled = best_scaler.transform(x_i)


        y_pred = dynamic_model.predict(x_i_scaled)[0]
        y_true = actual_cdirection.iloc[i]
        current_date = test_dates.iloc[i]


        correct = y_pred == y_true
        correct_count = correct_count + 1 if correct else correct_count
        cumulative_accuracy = correct_count / (i + 1)


        results["Date"].append(current_date)
        results["Real_CDirection"].append(y_true)
        results["Predicted_CDirection"].append(y_pred)
        results["Error_Rate"].append(1 - cumulative_accuracy)
        results["Cumulative_Accuracy"].append(cumulative_accuracy)


        X_walk = pd.concat([X_walk, x_i], ignore_index=True)
        y_walk = pd.concat([y_walk, pd.Series([y_true])], ignore_index=True)


        if (i + 1) % 5 == 0:
            X_walk_scaled = best_scaler.transform(X_walk)
            dynamic_model.fit(X_walk_scaled, y_walk)


    result_df = pd.DataFrame(results)
    print(f"\n{company_code} için walk-forward tamamlandı. Son doğruluk: {cumulative_accuracy:.2%}")
    return result_df'''

'import os\nimport glob\nimport numpy as np\nimport pandas as pd\nfrom typing import List, Tuple\nfrom sklearn.linear_model import SGDClassifier\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import TimeSeriesSplit\nfrom sklearn.metrics import log_loss\nfrom datetime import timedelta\n\n\nCONFIG = {\n    "window_size": 64,\n    "test_size": 64,\n    "data_paths": ["Datas/*.csv", "Test/*.csv"],\n    "start_date": "2006-07-27",\n    "end_date": "2025-05-06",\n    "n_splits": 12,\n}\n\ndef load_and_preprocess_single(file_path: str) -> pd.DataFrame:\n    try:\n        df = pd.read_csv(\n            file_path,\n            skiprows=2,\n            header=None,\n            names=["Date", "Close", "High", "Low", "Open", "Volume"]\n        )\n\n\n        df["Date"] = pd.to_datetime(df["Date"], errors=\'coerce\')\n        df = df.dropna(subset=["Date"])\n\n\n        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)\n        mask = (df["Date"] >=

In [ ]:
'''if __name__ == "__main__":
    result_dir = "Results/Change/Beast"
    os.makedirs(result_dir, exist_ok=True)

    company_files = sorted(glob.glob("Datas/*.csv"))
    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")

    for idx, filepath in enumerate(company_files):
        company_code = os.path.splitext(os.path.basename(filepath))[0]
        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")

        try:
            result_df = DRpredictor(company_code)

            if not result_df.empty:
                result_path = os.path.join(result_dir, f"{company_code}_results.csv")
                result_df.to_csv(result_path, index=False)
                print(f"→ Sonuçlar kaydedildi: {result_path}")
        except Exception as e:
            print(f"{company_code} işlenirken hata: {str(e)}")'''

'if __name__ == "__main__":\n    result_dir = "Results/Change/Beast"\n    os.makedirs(result_dir, exist_ok=True)\n\n    company_files = sorted(glob.glob("Datas/*.csv"))\n    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")\n\n    for idx, filepath in enumerate(company_files):\n        company_code = os.path.splitext(os.path.basename(filepath))[0]\n        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")\n\n        try:\n            result_df = DRpredictor(company_code)\n\n            if not result_df.empty:\n                result_path = os.path.join(result_dir, f"{company_code}_results.csv")\n                result_df.to_csv(result_path, index=False)\n                print(f"→ Sonuçlar kaydedildi: {result_path}")\n        except Exception as e:\n            print(f"{company_code} işlenirken hata: {str(e)}")'

In [ ]:
'''CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 12,
    "feature_columns": ["Close", "High", "Low", "Open", "Volume", "MA_20", "RSI", "Volatility"],
    "target_columns": ["CDirection", "Change"]
}

def load_and_preprocess_single(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(
            file_path,
            skiprows=2,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"]
        )

        # Tarih dönüşümü ve filtreleme
        df["Date"] = pd.to_datetime(df["Date"], errors='coerce', format='mixed')
        df = df.dropna(subset=["Date"])

        # Tarih aralığı filtresi
        mask = (
            (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) &
            (df["Date"] <= pd.to_datetime(CONFIG["end_date"]))
        )
        df = df.loc[mask]

        if df.empty:
            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")
            return pd.DataFrame()

        # Sayısal sütun dönüşümü
        numeric_cols = ["Close", "High", "Low", "Open", "Volume"]
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        df = df.dropna(subset=numeric_cols)

        return df

    except Exception as e:
        print(f"Hata: {file_path} yüklenirken - {str(e)}")
        return pd.DataFrame()

def calculate_features(df):
    df = df.copy()
    df["Change"] = df["Close"].pct_change(fill_method=None) * 100
    df["CDirection"] = np.where(df["Change"] < 0, 0, 1)


    df["MA_20"] = df["Close"].rolling(20, min_periods=1).mean()


    rsi = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
    df["RSI"] = rsi.fillna(50)


    volatility = df["Close"].pct_change().rolling(5, min_periods=1).std()
    df["Volatility"] = volatility.fillna(0.01)

    return df.dropna()

def load_company_data(company_code: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Dosya yollarını oluştur
    train_path = f"Datas/{company_code}.csv"
    test_path = f"Test/{company_code}.csv"

    # Verileri yükle
    train_df = load_and_preprocess_single(train_path)
    test_df = load_and_preprocess_single(test_path)

    if train_df.empty or test_df.empty:
        return pd.DataFrame(), pd.DataFrame()

    # Özellik mühendisliği
    train_df = calculate_features(train_df)
    test_df = calculate_features(test_df)

    return train_df, test_df

def prepare_data(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple:
    """Veriyi hazırlama fonksiyonu (hem yön hem de değişim için)"""
    # Tarih özellikleri
    def add_date_features(df):
        dt = df["Date"].dt
        return df.assign(
            Year=dt.year,
            Month=dt.month,
            Day=dt.day,
            Weekday=dt.weekday
        )

    df_train = add_date_features(df_train)
    df_test = add_date_features(df_test)

    # Özellikler ve hedefler
    feature_cols = [col for col in df_train.columns
                   if col not in ["Date", "Change", "CDirection"] and col in CONFIG["feature_columns"]]

    # Eğitim verisi
    X_train = df_train[feature_cols]
    y_train = df_train[CONFIG["target_columns"]]

    # Test verisi
    actual_values = df_test[CONFIG["target_columns"]].reset_index(drop=True)
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test[feature_cols]

    return X_train, y_train, actual_values, test_dates, X_test

def DRpredictor(company_code: str) -> pd.DataFrame:
    # Veri yükleme ve hazırlama
    train_df, test_df = load_company_data(company_code)
    if train_df.empty or test_df.empty:
        return pd.DataFrame()

    X_train, y_train, actual_values, test_dates, X_test = prepare_data(train_df, test_df)

    # Scaler'lar
    feature_scaler = StandardScaler()
    change_scaler = MinMaxScaler(feature_range=(-1, 1))

    # Veriyi ölçekle
    X_train_scaled = feature_scaler.fit_transform(X_train)
    y_change_scaled = change_scaler.fit_transform(y_train[["Change"]])

    # Çift hedef için veri hazırlama
    y_train_multi = np.column_stack((
        y_train["CDirection"].values,
        y_change_scaled.flatten()
    ))

    # Time Series Cross Validation
    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    best_val_loss = float('inf')
    best_model = None

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_scaled), 1):
        # Veriyi böl
        X_t, X_v = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_t, y_v = y_train_multi[train_idx], y_train_multi[val_idx]

        # Model oluştur (çift çıkışlı)
        model = MultiOutputRegressor(
            SGDRegressor(
                loss='huber',
                learning_rate='invscaling',
                eta0=0.01,
                power_t=0.25,
                early_stopping=True,
                validation_fraction=0.2,
                n_iter_no_change=5,
                random_state=42
            )
        )
        model.fit(X_t, y_t)

        # Validasyon hatası
        y_pred = model.predict(X_v)
        val_loss = mean_squared_error(y_v, y_pred)

        # En iyi modeli güncelle
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model)

    print(f"En iyi model val_loss: {best_val_loss:.4f}")

    # Walk-forward test süreci
    results = {
        "Date": test_dates.tolist(),
        "Real_CDirection": actual_values["CDirection"].tolist(),
        "Predicted_CDirection": [],
        "Real_Change": actual_values["Change"].tolist(),
        "Predicted_Change": [],
        "Direction_Accuracy": [],
        "Change_RMSE": []
    }

    # Online öğrenme için model
    online_model = deepcopy(best_model)
    X_walk = X_train.copy()
    y_walk = y_train.copy()

    # Doğru yön sayacı
    correct_direction_count = 0

    for i in range(len(X_test)):
        # Test örneğini ölçeklendir
        x_i = X_test.iloc[[i]]
        x_i_scaled = feature_scaler.transform(x_i)

        # Tahmin yap
        y_pred_scaled = online_model.predict(x_i_scaled)[0]

        # Çıktıları dönüştür
        direction_pred = 1 if y_pred_scaled[0] > 0.5 else 0
        change_pred = change_scaler.inverse_transform([[y_pred_scaled[1]]])[0][0]

        # Gerçek değerler
        direction_true = actual_values["CDirection"].iloc[i]
        change_true = actual_values["Change"].iloc[i]

        # Yön doğruluğu
        correct_direction = direction_pred == direction_true
        correct_direction_count = correct_direction_count + 1 if correct_direction else correct_direction_count
        direction_accuracy = correct_direction_count / (i + 1)

        # Değişim hatası (RMSE)
        current_rmse = np.sqrt(mean_squared_error(
            actual_values["Change"].iloc[:i+1],
            results["Predicted_Change"] + [change_pred]
        )) if i > 0 else 0

        # Sonuçları kaydet
        results["Predicted_CDirection"].append(direction_pred)
        results["Predicted_Change"].append(change_pred)
        results["Direction_Accuracy"].append(direction_accuracy)
        results["Change_RMSE"].append(current_rmse)

        # Modeli güncelle (online learning)
        X_walk = pd.concat([X_walk, x_i], ignore_index=True)
        y_walk = pd.concat([
            y_walk,
            pd.DataFrame([[direction_true, change_true]], columns=CONFIG["target_columns"])
        ], ignore_index=True)

        # Her 5 adımda bir modeli yeniden eğit
        if (i + 1) % 5 == 0:
            X_walk_scaled = feature_scaler.transform(X_walk)
            y_change_scaled = change_scaler.transform(y_walk[["Change"]])
            y_walk_multi = np.column_stack((
                y_walk["CDirection"].values,
                y_change_scaled.flatten()
            ))
            online_model.fit(X_walk_scaled, y_walk_multi)

    # Sonuçları DataFrame'e dönüştür
    result_df = pd.DataFrame(results)
    final_acc = accuracy_score(
        result_df["Real_CDirection"],
        result_df["Predicted_CDirection"]
    )
    final_rmse = np.sqrt(mean_squared_error(
        result_df["Real_Change"],
        result_df["Predicted_Change"]
    ))

    print(f"{company_code} sonuçları: Yön Doğruluğu: {final_acc:.2%}, Change RMSE: {final_rmse:.4f}")
    return result_df'''

'CONFIG = {\n    "window_size": 64,\n    "test_size": 64,\n    "data_paths": ["Datas/*.csv", "Test/*.csv"],\n    "start_date": "2006-07-27",\n    "end_date": "2025-05-06",\n    "n_splits": 12,\n    "feature_columns": ["Close", "High", "Low", "Open", "Volume", "MA_20", "RSI", "Volatility"],\n    "target_columns": ["CDirection", "Change"]\n}\n\ndef load_and_preprocess_single(file_path: str) -> pd.DataFrame:\n    try:\n        df = pd.read_csv(\n            file_path,\n            skiprows=2,\n            header=None,\n            names=["Date", "Close", "High", "Low", "Open", "Volume"]\n        )\n\n        # Tarih dönüşümü ve filtreleme\n        df["Date"] = pd.to_datetime(df["Date"], errors=\'coerce\', format=\'mixed\')\n        df = df.dropna(subset=["Date"])\n\n        # Tarih aralığı filtresi\n        mask = (\n            (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) &\n            (df["Date"] <= pd.to_datetime(CONFIG["end_date"]))\n        )\n        df = df.loc[mask]\n\n  

In [ ]:
'''# Ana işlem
if __name__ == "__main__":
    result_dir = "Results/Change/Beast"
    os.makedirs(result_dir, exist_ok=True)

    company_files = sorted(glob.glob("Datas/*.csv"))
    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")

    for idx, filepath in enumerate(company_files):
        company_code = os.path.splitext(os.path.basename(filepath))[0]
        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")

        try:
            result_df = DRpredictor(company_code)
            if not result_df.empty:
                result_path = os.path.join(result_dir, f"{company_code}_results.csv")
                result_df.to_csv(result_path, index=False)
                print(f"→ Sonuçlar kaydedildi: {result_path}")
        except Exception as e:
            print(f"{company_code} işlenirken hata: {str(e)}")'''

'# Ana işlem\nif __name__ == "__main__":\n    result_dir = "Results/Change/Beast"\n    os.makedirs(result_dir, exist_ok=True)\n\n    company_files = sorted(glob.glob("Datas/*.csv"))\n    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")\n\n    for idx, filepath in enumerate(company_files):\n        company_code = os.path.splitext(os.path.basename(filepath))[0]\n        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")\n\n        try:\n            result_df = DRpredictor(company_code)\n            if not result_df.empty:\n                result_path = os.path.join(result_dir, f"{company_code}_results.csv")\n                result_df.to_csv(result_path, index=False)\n                print(f"→ Sonuçlar kaydedildi: {result_path}")\n        except Exception as e:\n            print(f"{company_code} işlenirken hata: {str(e)}")'

In [ ]:
'''CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 12,
}

def load_and_preprocess_single(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(
            file_path,
            skiprows=2,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"]
        )

        df["Date"] = pd.to_datetime(df["Date"], errors='coerce', format='mixed')
        df = df.dropna(subset=["Date"])

        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)
        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)
        df = df.loc[mask]

        if df.empty:
            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")
            return pd.DataFrame()

        for col in ["Close", "High", "Low", "Open", "Volume"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        return df.dropna()

    except Exception as e:
        print(f"Hata: {file_path} yüklenirken - {str(e)}")
        return pd.DataFrame()

def load_company_data(company_code: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_path = f"Datas/{company_code}.csv"
    test_path = f"Test/{company_code}.csv"

    train_df = load_and_preprocess_single(train_path)
    test_df = load_and_preprocess_single(test_path)

    if train_df.empty or test_df.empty:
        return pd.DataFrame(), pd.DataFrame()

    def calculate_features(df):
        df = df.copy()
        # Önceki güne göre yüzde değişim
        df["Change"] = df["Close"].pct_change(fill_method=None) * 100

        # Özellik mühendisliği
        df["MA_20"] = df["Close"].rolling(20, min_periods=1).mean()

        # RSI hesaplama
        rsi = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
        df["RSI"] = rsi.fillna(50)

        # Volatilite
        volatility = df["Close"].pct_change().rolling(5, min_periods=1).std()
        df["Volatility"] = volatility.fillna(0.01)

        # Önceki günün kapanışı
        df["Prev_Close"] = df["Close"].shift(1)

        return df.dropna()

    train_df = calculate_features(train_df)
    test_df = calculate_features(test_df)

    return train_df, test_df

def dater(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple:
    def process_date_features(df):
        df = df.copy()
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Month"] = df["Date"].dt.month
        df["Day"] = df["Date"].dt.day
        df["Weekday"] = df["Date"].dt.weekday
        return df

    df_train = process_date_features(df_train)
    df_test = process_date_features(df_test)

    # Hedef değişken ve özellikler
    y_train = df_train["Change"]
    # Gereksiz sütunları kaldır
    X_train = df_train.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    actual_change = df_test["Change"].reset_index(drop=True)
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    return X_train, y_train, actual_change, test_dates, X_test

def DRpredictor(company_code: str) -> pd.DataFrame:
    train_df, test_df = load_company_data(company_code)

    if train_df.empty or test_df.empty:
        print(f"{company_code} için yeterli veri yok. Atlanıyor.")
        return pd.DataFrame()

    X_train, y_train, actual_change, test_dates, X_test = dater(train_df, test_df)

    # Hacim sütununu log dönüşümü
    for df in [X_train, X_test]:
        if "Volume" in df.columns:
            df["Volume"] = np.log1p(df["Volume"])

    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    best_val_loss = float('inf')
    best_scaler = None
    best_model = None

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
        X_t, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]

        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_t_scaled = scaler.fit_transform(X_t)
        X_v_scaled = scaler.transform(X_v)

        model = SGDRegressor(
            alpha=0.0001,  # Regularizasyon
            learning_rate='invscaling',
            eta0=0.001,  # Düşük öğrenme oranı
            max_iter=1000,
            tol=1e-3,
            random_state=42,
            early_stopping=False,
            shuffle=False
        )
        model.fit(X_t_scaled, y_t)

        y_pred_val = model.predict(X_v_scaled)
        val_loss = mean_squared_error(y_v, y_pred_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_scaler = scaler
            best_model = model

    print(f"\nEn iyi model val_loss (MSE): {best_val_loss:.6f}")

    # Sonuçları saklamak için yapı
    results = {
        "Date": [],
        "Real_Change": [],
        "Predicted_Change": [],
        "Price_Error": [],
        "Squared_Error": [],
        "Cumulative_MSE": []
    }

    total_squared_error = 0
    X_walk = X_train.copy()
    y_walk = y_train.copy()

    dynamic_model = HuberRegressor(
        alpha=0.0001,
        max_iter=1000,
        warm_start=True
    )

    # İlk modeli eğit
    X_walk_scaled = best_scaler.transform(X_walk)
    dynamic_model.fit(X_walk_scaled, y_walk)

    for i in range(20):
        x_i = X_test.iloc[[i]]
        x_i_scaled = best_scaler.transform(x_i)

        y_pred = best_model.predict(x_i_scaled)[0]
        y_true = actual_change.iloc[i]
        current_date = test_dates.iloc[i]

        price_error = y_true - y_pred
        squared_error = (y_true - y_pred) ** 2
        total_squared_error += squared_error
        cumulative_mse = total_squared_error / (i + 1)

        results["Date"].append(current_date)
        results["Real_Change"].append(y_true)
        results["Predicted_Change"].append(y_pred)
        results["Price_Error"].append(price_error)
        results["Squared_Error"].append(squared_error)
        results["Cumulative_MSE"].append(cumulative_mse)

        # Modeli güncelle
        X_walk = pd.concat([X_walk, x_i], ignore_index=True)
        y_walk = pd.concat([y_walk, pd.Series([y_true])], ignore_index=True)

        # Her 5 adımda bir modeli yeniden eğit
        if (i + 1) % 5 == 0:
            X_walk_scaled = best_scaler.transform(X_walk)
            best_model.fit(X_walk_scaled, y_walk)

    result_df = pd.DataFrame(results)
    print(f"\n{company_code} için walk-forward tamamlandı. Son MSE: {cumulative_mse:.6f}")
    return result_df'''

'CONFIG = {\n    "window_size": 64,\n    "test_size": 64,\n    "data_paths": ["Datas/*.csv", "Test/*.csv"],\n    "start_date": "2006-07-27",\n    "end_date": "2025-05-06",\n    "n_splits": 12,\n}\n\ndef load_and_preprocess_single(file_path: str) -> pd.DataFrame:\n    try:\n        df = pd.read_csv(\n            file_path,\n            skiprows=2,\n            header=None,\n            names=["Date", "Close", "High", "Low", "Open", "Volume"]\n        )\n\n        df["Date"] = pd.to_datetime(df["Date"], errors=\'coerce\', format=\'mixed\')\n        df = df.dropna(subset=["Date"])\n\n        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)\n        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)\n        df = df.loc[mask]\n\n        if df.empty:\n            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")\n            return pd.DataFrame()\n\n        for col in ["Close", "High", "Low", "Open", "Volume"]:

In [ ]:
'''if __name__ == "__main__":
    result_dir = "Results/Change/Beast"
    os.makedirs(result_dir, exist_ok=True)

    company_files = sorted(glob.glob("Datas/*.csv"))
    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")

    for idx, filepath in enumerate(company_files):
        company_code = os.path.splitext(os.path.basename(filepath))[0]
        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")

        try:
            result_df = DRpredictor(company_code)

            if not result_df.empty:
                result_path = os.path.join(result_dir, f"{company_code}_results.csv")
                result_df.to_csv(result_path, index=False)
                print(f"→ Sonuçlar kaydedildi: {result_path}")
        except Exception as e:
            print(f"{company_code} işlenirken hata: {str(e)}")'''

'if __name__ == "__main__":\n    result_dir = "Results/Change/Beast"\n    os.makedirs(result_dir, exist_ok=True)\n\n    company_files = sorted(glob.glob("Datas/*.csv"))\n    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")\n\n    for idx, filepath in enumerate(company_files):\n        company_code = os.path.splitext(os.path.basename(filepath))[0]\n        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")\n\n        try:\n            result_df = DRpredictor(company_code)\n\n            if not result_df.empty:\n                result_path = os.path.join(result_dir, f"{company_code}_results.csv")\n                result_df.to_csv(result_path, index=False)\n                print(f"→ Sonuçlar kaydedildi: {result_path}")\n        except Exception as e:\n            print(f"{company_code} işlenirken hata: {str(e)}")'

In [ ]:
'''CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 15,
}

def load_and_preprocess_single(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(
            file_path,
            skiprows=2,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"]
        )

        df["Date"] = pd.to_datetime(df["Date"], errors='coerce', format="mixed")
        df = df.dropna(subset=["Date"])

        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)
        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)
        df = df.loc[mask]

        if df.empty:
            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")
            return pd.DataFrame()

        for col in ["Close", "High", "Low", "Open", "Volume"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        return df.dropna()

    except Exception as e:
        print(f"Hata: {file_path} yüklenirken - {str(e)}")
        return pd.DataFrame()

def load_company_data(company_code: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_path = f"Datas/{company_code}.csv"
    test_path = f"Test/{company_code}.csv"

    train_df = load_and_preprocess_single(train_path)
    test_df = load_and_preprocess_single(test_path)

    if train_df.empty or test_df.empty:
        return pd.DataFrame(), pd.DataFrame()

    def calculate_features(df):
        df = df.copy()
        df["Change"] = df["Close"].pct_change(fill_method=None) * 100

        # Özellik mühendisliği
        df["MA_20"] = df["Close"].rolling(20, min_periods=1).mean()

        # RSI hesaplama
        rsi = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
        df["RSI"] = rsi.fillna(50)

        # Volatilite
        volatility = df["Close"].pct_change().rolling(5, min_periods=1).std()
        df["Volatility"] = volatility.fillna(0.01)

        # Önceki günün kapanışı
        df["Prev_Close"] = df["Close"].shift(1)

        return df.dropna()

    train_df = calculate_features(train_df)
    test_df = calculate_features(test_df)

    return train_df, test_df

def dater(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple:
    def process_date_features(df):
        df = df.copy()
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Month"] = df["Date"].dt.month
        df["Day"] = df["Date"].dt.day
        df["Weekday"] = df["Date"].dt.weekday
        return df

    df_train = process_date_features(df_train)
    df_test = process_date_features(df_test)

    # Hedef değişken ve özellikler
    y_train = df_train["Change"]
    # Gereksiz sütunları kaldır
    X_train = df_train.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    actual_change = df_test["Change"].reset_index(drop=True)
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    return X_train, y_train, actual_change, test_dates, X_test

def DRpredictor(company_code: str) -> pd.DataFrame:
    train_df, test_df = load_company_data(company_code)

    if train_df.empty or test_df.empty:
        print(f"{company_code} için yeterli veri yok. Atlanıyor.")
        return pd.DataFrame()

    X_train, y_train, actual_change, test_dates, X_test = dater(train_df, test_df)

    # Hacim sütununu log dönüşümü
    for df in [X_train, X_test]:
        if "Volume" in df.columns:
            df["Volume"] = np.log1p(df["Volume"])

    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    best_val_loss = float('inf')
    best_scaler = None
    best_model = None

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
        X_t, X_v = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]

        scaler = StandardScaler()
        X_t_scaled = scaler.fit_transform(X_t)
        X_v_scaled = scaler.transform(X_v)

        model = SGDRegressor(
            alpha=0.0001,
            learning_rate='invscaling',
            eta0=0.001,
            max_iter=1000,
            tol=1e-3,
            random_state=42,
            early_stopping=False
        )
        model.fit(X_t_scaled, y_t)

        y_pred_val = model.predict(X_v_scaled)
        val_loss = mean_squared_error(y_v, y_pred_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_scaler = scaler
            best_model = model

    print(f"\nEn iyi model val_loss (MSE): {best_val_loss:.6f}")

    # 1. BEST_MODEL'İ TÜM EĞİTİM VERİSİYLE YENİDEN EĞİT
    X_train_full_scaled = best_scaler.transform(X_train)
    best_model_full = SGDRegressor(
        **best_model.get_params()  # Aynı hiperparametreleri kullan
    )
    best_model_full.fit(X_train_full_scaled, y_train)

    # 2. DEEPCOPY İLE DYNAMIC_MODEL OLUŞTUR
    dynamic_model = copy.deepcopy(best_model_full)
    dynamic_model.warm_start = True  # Artımlı öğrenme için

    # Sonuçları saklamak için yapı
    results = {
        "Date": [],
        "Real_Change": [],
        "Predicted_Change": [],
        "Squared_Error": [],
        "Cumulative_MSE": []
    }

    total_squared_error = 0
    X_walk = X_train.copy()
    y_walk = y_train.copy()

    # 3. İLK DURUMDA DYNAMIC_MODEL ZATEN EĞİTİLMİŞ DURUMDA
    for i in range(len(X_test)):
        x_i = X_test.iloc[[i]]
        x_i_scaled = best_scaler.transform(x_i)

        # DYNAMIC_MODEL ile tahmin yap
        y_pred = dynamic_model.predict(x_i_scaled)[0]
        y_true = actual_change.iloc[i]
        current_date = test_dates.iloc[i]

        squared_error = (y_true - y_pred) ** 2
        total_squared_error += squared_error
        cumulative_mse = total_squared_error / (i + 1)

        results["Date"].append(current_date)
        results["Real_Change"].append(y_true)
        results["Predicted_Change"].append(y_pred)
        results["Squared_Error"].append(squared_error)
        results["Cumulative_MSE"].append(cumulative_mse)

        # Modeli güncelle
        X_walk = pd.concat([X_walk, x_i], ignore_index=True)
        y_walk = pd.concat([y_walk, pd.Series([y_true])], ignore_index=True)

        # Her 5 adımda bir modeli yeniden eğit
        if (i + 1) % 5 == 0:
            X_walk_scaled = best_scaler.transform(X_walk)
            dynamic_model.fit(X_walk_scaled, y_walk)

    result_df = pd.DataFrame(results)
    print(f"\n{company_code} için walk-forward tamamlandı. Son MSE: {cumulative_mse:.6f}")
    return result_df'''

'CONFIG = {\n    "window_size": 64,\n    "test_size": 64,\n    "data_paths": ["Datas/*.csv", "Test/*.csv"],\n    "start_date": "2006-07-27",\n    "end_date": "2025-05-06",\n    "n_splits": 15,\n}\n\ndef load_and_preprocess_single(file_path: str) -> pd.DataFrame:\n    try:\n        df = pd.read_csv(\n            file_path,\n            skiprows=2,\n            header=None,\n            names=["Date", "Close", "High", "Low", "Open", "Volume"]\n        )\n\n        df["Date"] = pd.to_datetime(df["Date"], errors=\'coerce\', format="mixed")\n        df = df.dropna(subset=["Date"])\n\n        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)\n        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)\n        df = df.loc[mask]\n\n        if df.empty:\n            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")\n            return pd.DataFrame()\n\n        for col in ["Close", "High", "Low", "Open", "Volume"]:\n

In [ ]:
'''if __name__ == "__main__":
    result_dir = "Results/Change/Beast"
    os.makedirs(result_dir, exist_ok=True)

    company_files = sorted(glob.glob("Datas/*.csv"))
    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")

    for idx, filepath in enumerate(company_files):
        company_code = os.path.splitext(os.path.basename(filepath))[0]
        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")

        try:
            result_df = DRpredictor(company_code)

            if not result_df.empty:
                result_path = os.path.join(result_dir, f"{company_code}_results.csv")
                result_df.to_csv(result_path, index=False)
                print(f"→ Sonuçlar kaydedildi: {result_path}")
        except Exception as e:
            print(f"{company_code} işlenirken hata: {str(e)}")'''

'if __name__ == "__main__":\n    result_dir = "Results/Change/Beast"\n    os.makedirs(result_dir, exist_ok=True)\n\n    company_files = sorted(glob.glob("Datas/*.csv"))\n    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")\n\n    for idx, filepath in enumerate(company_files):\n        company_code = os.path.splitext(os.path.basename(filepath))[0]\n        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")\n\n        try:\n            result_df = DRpredictor(company_code)\n\n            if not result_df.empty:\n                result_path = os.path.join(result_dir, f"{company_code}_results.csv")\n                result_df.to_csv(result_path, index=False)\n                print(f"→ Sonuçlar kaydedildi: {result_path}")\n        except Exception as e:\n            print(f"{company_code} işlenirken hata: {str(e)}")'

In [22]:
from typing_extensions import final
CONFIG = {
    "window_size": 64,
    "test_size": 64,
    "data_paths": ["Datas/*.csv", "Test/*.csv"],
    "start_date": "2006-07-27",
    "end_date": "2025-05-06",
    "n_splits": 12,
}

def load_and_preprocess_single(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(
            file_path,
            skiprows=2,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"]
        )

        df["Date"] = pd.to_datetime(df["Date"], errors='coerce', format='mixed')
        df = df.dropna(subset=["Date"])

        end_date = pd.to_datetime(CONFIG["end_date"]) + timedelta(days=1)
        mask = (df["Date"] >= pd.to_datetime(CONFIG["start_date"])) & (df["Date"] <= end_date)
        df = df.loc[mask]

        if df.empty:
            print(f"Uyarı: {os.path.basename(file_path)} dosyası filtreleme sonrası boş")
            return pd.DataFrame()

        for col in ["Close", "High", "Low", "Open", "Volume"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        return df.dropna()

    except Exception as e:
        print(f"Hata: {file_path} yüklenirken - {str(e)}")
        return pd.DataFrame()

def load_company_data(company_code: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_path = f"Datas/{company_code}.csv"
    test_path = f"Test/{company_code}.csv"

    train_df = load_and_preprocess_single(train_path)
    test_df = load_and_preprocess_single(test_path)

    if train_df.empty or test_df.empty:
        return pd.DataFrame(), pd.DataFrame()

    def calculate_features(df):
        df = df.copy()
        df["Change"] = df["Close"].pct_change(fill_method=None) * 100

        # Özellik mühendisliği
        df["MA_20"] = df["Close"].rolling(20, min_periods=1).mean()

        # RSI hesaplama
        rsi = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
        df["RSI"] = rsi.fillna(50)

        # Volatilite
        volatility = df["Close"].pct_change().rolling(5, min_periods=1).std()
        df["Volatility"] = volatility.fillna(0.01)

        return df.dropna()

    train_df = calculate_features(train_df)
    test_df = calculate_features(test_df)

    return train_df, test_df

def dater(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple:
    def process_date_features(df):
        df = df.copy()
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        df["Month"] = df["Date"].dt.month
        df["Day"] = df["Date"].dt.day
        df["Weekday"] = df["Date"].dt.weekday
        return df

    df_train = process_date_features(df_train)
    df_test = process_date_features(df_test)

    y_train = df_train["Change"]
    X_train = df_train.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    actual_change = df_test["Change"].reset_index(drop=True)
    test_dates = df_test["Date"].reset_index(drop=True)
    X_test = df_test.drop(["Date", "Change", "CDirection"], axis=1, errors="ignore")

    return X_train, y_train, actual_change, test_dates, X_test

def DRpredictor(company_code: str) -> pd.DataFrame:
    train_df, test_df = load_company_data(company_code)

    if train_df.empty or test_df.empty:
        print(f"{company_code} için yeterli veri yok. Atlanıyor.")
        return pd.DataFrame()

    X_train, y_train, actual_change, test_dates, X_test = dater(train_df, test_df)

    scaler = RobustScaler(quantile_range=(5, 95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)


    tscv = TimeSeriesSplit(n_splits=CONFIG["n_splits"])
    best_val_loss = float('inf')
    best_model = None

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_scaled), 1):
        X_t, X_v = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_t, y_v = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = HuberRegressor(
            alpha=0.001,
            epsilon=1.5,
            max_iter=1000,
            tol=1e-4
        )
        model.fit(X_t, y_t)

        y_pred_val = model.predict(X_v)
        val_loss = mean_squared_error(y_v, y_pred_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

    print(f"\nEn iyi model val_loss (MSE): {best_val_loss:.6f}")

    best_model.fit(X_train_scaled, y_train)

    results = {
        "Date": [],
        "Real_Change": [],
        "Predicted_Change": [],
        "Squared_Error": [],
        "Cumulative_MSE": []
    }

    total_squared_error = 0
    X_walk = X_train_scaled.copy()
    y_walk = y_train.copy().values

    for i in range(len(X_test_scaled)):
        x_i = X_test_scaled[i].reshape(1, -1)

        y_pred = best_model.predict(x_i)[0]
        y_true = actual_change.iloc[i]
        current_date = test_dates.iloc[i]

        squared_error = (y_true - y_pred) ** 2
        total_squared_error += squared_error
        cumulative_mse = total_squared_error / (i + 1)

        results["Date"].append(current_date)
        results["Real_Change"].append(y_true)
        results["Predicted_Change"].append(y_pred)
        results["Squared_Error"].append(squared_error)
        results["Cumulative_MSE"].append(cumulative_mse)

        X_walk = np.vstack([X_walk, x_i])
        y_walk = np.append(y_walk, y_true)

        best_model.fit(X_walk, y_walk)

    result_df = pd.DataFrame(results)
    final_mse = cumulative_mse
    print(f"\n{company_code} için walk-forward tamamlandı. Son MSE: {final_mse:.6f}")

    return result_df

In [23]:
if __name__ == "__main__":
    result_dir = "Results/Change/Beast"
    os.makedirs(result_dir, exist_ok=True)

    company_files = sorted(glob.glob("Datas/*.csv"))
    print(f"İşlenecek toplam şirket sayısı: {len(company_files)}")

    for idx, filepath in enumerate(company_files):
        company_code = os.path.splitext(os.path.basename(filepath))[0]
        print(f"\n[{idx+1}/{len(company_files)}] {company_code} işleniyor...")

        try:
            result_df = DRpredictor(company_code)

            if not result_df.empty:
                result_path = os.path.join(result_dir, f"{company_code}_results.csv")
                result_df.to_csv(result_path, index=False)
                print(f"→ Sonuçlar kaydedildi: {result_path}")
        except Exception as e:
            print(f"{company_code} işlenirken hata: {str(e)}")

İşlenecek toplam şirket sayısı: 30

[1/30] AKBNK işleniyor...

En iyi model val_loss (MSE): 0.651192

AKBNK için walk-forward tamamlandı. Son MSE: 3.258560
→ Sonuçlar kaydedildi: Results/Change/Beast/AKBNK_results.csv

[2/30] ALARK işleniyor...

En iyi model val_loss (MSE): 2.142265

ALARK için walk-forward tamamlandı. Son MSE: 2.562204
→ Sonuçlar kaydedildi: Results/Change/Beast/ALARK_results.csv

[3/30] ARCLK işleniyor...

En iyi model val_loss (MSE): 0.727653

ARCLK için walk-forward tamamlandı. Son MSE: 1.796371
→ Sonuçlar kaydedildi: Results/Change/Beast/ARCLK_results.csv

[4/30] ASELS işleniyor...

En iyi model val_loss (MSE): 1.345815

ASELS için walk-forward tamamlandı. Son MSE: 10.327622
→ Sonuçlar kaydedildi: Results/Change/Beast/ASELS_results.csv

[5/30] BIMAS işleniyor...

En iyi model val_loss (MSE): 0.969401

BIMAS için walk-forward tamamlandı. Son MSE: 2.328013
→ Sonuçlar kaydedildi: Results/Change/Beast/BIMAS_results.csv

[6/30] BRSAN işleniyor...

En iyi model val_loss